In [73]:
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

import requests

In [152]:
df = pd.read_excel("C:/Python_practice/DL_project/gwangjin_contest/land_crawling_data.xlsx", sheet_name="크롤링")

In [153]:
df = df[19343:]
df

,행정동,본번,부번,도로명주소,도로명주소(크롤링),토지면적,주용도(건물유형),공시지가_기준년월,공시지가
19343,중곡동,601,12,서울특별시 광진구 용마산로36길 22-6 (중곡동),NaN,NaN,NaN,NaN,NaN
19344,중곡동,601,13,서울특별시 광진구 용마산로36길 22 (중곡동),NaN,NaN,NaN,NaN,NaN
19345,중곡동,601,2,"서울특별시 광진구 용마산로36길 22-5 (중곡동, 리치팰리스)",NaN,NaN,NaN,NaN,NaN
19346,중곡동,601,4,서울특별시 광진구 용마산로36길 22-9 (중곡동),NaN,NaN,NaN,NaN,NaN
19347,중곡동,601,5,서울특별시 광진구 용마산로36길 22-11 (중곡동),NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
23239,화양동,9,89,"서울특별시 광진구 아차산로29길 18 (화양동, 화양동근생)",NaN,NaN,NaN,NaN,NaN
23240,화양동,99,0,서울특별시 광진구 군자로6길 45 (화양동),NaN,NaN,NaN,NaN,NaN
23241,화양동,9,9,서울특별시 광진구 아차산로31길 19 (화양동),NaN,NaN,NaN,NaN,NaN
23242,화양동,9,90,서울특별시 광진구 아차산로29길 12 (화양동),NaN,NaN,NaN,NaN,NaN


In [154]:
# 접속 url
url = "https://land.gwangjin.go.kr:449/land/wskras/generalInfo.do#"

# 주소명 검색 홈페이지 접속
path = 'C:/Python_practice/DL_project/music_resting_place_ver_dh/0. dataset/chromedriver_win32/chromedriver.exe'
browser = Chrome(executable_path=path) # 개인에게 맞는 경로 수정 필요
browser.get(url)

for index, row in df.iterrows():
    # 법정동 선택
    Select(browser.find_element(By.ID, "selBjdong")).select_by_visible_text(row['행정동'])
    time.sleep(1)

    # 본번 입력
    browser.find_element(By.ID, "bonbeon").send_keys(row['본번'])
    time.sleep(1)

    # 부번 입력
    browser.find_element(By.ID, "bubeon").send_keys(row['부번'])
    time.sleep(1)

    # 검색 버튼 클릭
    browser.find_element(By.ID, "btnSearch").click()
    time.sleep(2)
    
    # 각 요소가 로드될 때까지 대기
    wait = WebDriverWait(browser, 4)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#rdVal2")))
    # wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#kras11Ym")))
    # wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#kras11Jiga")))

    # BeautifulSoup로 페이지 파싱
    soup = BeautifulSoup(browser.page_source, "html.parser")

    # 도로명주소
    address_type_element = soup.select_one("#newAddressInfo option")
    no_address_type_elements = browser.find_elements(By.CSS_SELECTOR, "#newAddressInfo")

    if address_type_element and address_type_element.text:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#newAddressInfo option")))
        df.loc[index, '도로명주소(크롤링)'] = address_type_element.text
    elif no_address_type_elements and no_address_type_elements[0].text == "" :
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#newAddressInfo")))
        df.loc[index, '도로명주소(크롤링)'] = np.nan
    elif address_type_element and address_type_element.text == "도로명주소 정보가 없습니다" :
        df.loc[index, '도로명주소(크롤링)'] = np.nan


    # 토지면적
    df.loc[index, '토지면적'] = soup.select_one("#rdVal2").text

    # 주용도(건물유형)
    building_type_element = soup.select_one("#tblBuildListTab0 tbody tr td div.nowrap")
    no_building_data_elements = browser.find_elements(By.CSS_SELECTOR, "#tblBuildListTab0 tbody tr td[id='bldrgstListTab']")

    if building_type_element and building_type_element.text:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#tblBuildListTab0 tbody tr td div.nowrap")))
        df.loc[index, '주용도(건물유형)'] = building_type_element.text
    elif no_building_data_elements and no_building_data_elements[0].text == "조회된 내용이 없습니다.":
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#tblBuildListTab0 tbody tr td[id='bldrgstListTab']")))
        df.loc[index, '주용도(건물유형)'] = np.nan


    # 공시지가_기준년월
    date_type_element = soup.select_one("#kras11Ym")
    no_land_price_type_element = browser.find_elements(By.CSS_SELECTOR, "#kras11")

    if date_type_element and date_type_element.text:
        df.loc[index, '공시지가_기준년월'] = date_type_element.text
    elif no_land_price_type_element and no_land_price_type_element[0].text == "공시지가 확인서 요청시 요청한 주소에 대한 지가정보가 없습니다.":
        df.loc[index, '공시지가'] = np.nan
    


    # 공시지가
    land_price_type_element = soup.select_one("#kras11Jiga")
    no_land_price_type_element = browser.find_elements(By.CSS_SELECTOR, "#kras11")

    if land_price_type_element and land_price_type_element.text:
        df.loc[index, '공시지가'] = land_price_type_element.text
    elif no_land_price_type_element and no_land_price_type_element[0].text == "공시지가 확인서 요청시 요청한 주소에 대한 지가정보가 없습니다.":
        df.loc[index, '공시지가'] = np.nan

    # 본번 및 부번 입력란 초기화
    browser.find_element(By.ID, "bonbeon").clear()
    browser.find_element(By.ID, "bubeon").clear()
    time.sleep(1)

# 드라이버 종료
browser.quit()

# # 저장
df.to_excel("C:/Python_practice/DL_project/gwangjin_contest/land_crawling_data_completed_v4.xlsx", index=False)

C:\Users\dohee\AppData\Local\Temp\ipykernel_17604\739205099.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = Chrome(executable_path=path) # 개인에게 맞는 경로 수정 필요


In [70]:
# 0 ~ 211 
# df.to_excel("C:/Python_practice/DL_project/gwangjin_contest/land_crawling_data_completed.xlsx", index=False)

In [150]:
# # 저장
df.to_excel("C:/Python_practice/DL_project/gwangjin_contest/land_crawling_data_completed_v2.xlsx", index=False)

<hr>

웹 정보를 가져오는 방식 2가지<br>
[1] 웹 API를 사용하여 정보를 가져오는 방식<br>
웹 API는 서버와 클라이언트 간에 정보를 주고받는 통신 규약이며, 일반적으로 JSON 또는 XML 형식으로 데이터를 제공<br>
<br>
[2] 웹스크랩 = 크롤링<br>
웹 페이지의 HTML 구조에서 필요한 정보를 추출하는 과정<br>

웹 API를 사용 권장<br>
서버가 제공하는 데이터에 직접적으로 접근하여 필요한 정보를 가져올 수 있어<br>
코드 작성이 더 간단해지고 효율성이 높아짐

In [77]:
# # 웹 API를 사용하여 정보를 가져오는 방식
# 보안이 되어 있어 api 요청 및 추출이 되지 않아 크롤링으로 진행

# url = "https://land.gwangjin.go.kr:449/land/common/getRoadAddrList.do"
# params = {'address': '광진구 광장동 394-43',
#               'bjdongCd': '',
#     'page': '1',
#     'rows': '10',
#     'searchYn': 'Y'
#     }

# # 요청 보내기
# response = requests.get(url, params=params)

# # JSON 데이터에서 "rnAddr" 추출
# json_data = response.json()
# rn_addr = json_data["result"][0]["rnAddr"]

# print(rn_addr)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)